In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
#################################################################
def Vh2V3k(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    v3k = Vh-25.2*cosl*cosb-245.7*sinl*cosb+276.8*sinb

    return v3k

#################################################################

def Vcmb2Vmod(Vcmb, omegam=0.27):
    
    omegal=1.-omegam
    c=299800.
    z=Vcmb/c
    q0=0.5*(omegam-2.*omegal)
    fmod=1.+0.5*(1.-q0)*z-(1./6)*(1.-q0-3.*q0**2+1.)*z**2
    Vmod=c*z*fmod

    return Vmod
#################################################################

def Vh2Vls(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    vls = Vh-26.*cosl*cosb+317.*sinl*cosb-8.*sinb

    return vls
#################################################################

In [3]:
df = pd.read_csv('TFR_CF4_components_bar', delimiter='|')
df = df.rename(columns=lambda x: x.strip())

df_TFR = df.set_index('PGC')
df_TFR = df_TFR.rename(columns={"Nest": "Nest_TFR"})
df_TFR = df_TFR.rename(columns={"PGC1": "PGC1_TFR"})
df_TFR = df_TFR.rename(columns={"Vcmb": "Vcmb_TFR"})
df_TFR = df_TFR.rename(columns={"DM_av": "DM_TFR"})
df_TFR = df_TFR.rename(columns={"eDM_av": "eDM_TFR"})

df_TFR = df_TFR[df_TFR["av_flag"]==0]

df_TFR.head()

,PGC1_TFR,Nest_TFR,Vcmb_TFR,Vmod,DM_TFR,eDM_TFR,av_flag,DM_cf4,eDM_cf4,cf4_flag,...,cf2_flag,DM_sfi,eDM_sfi,sfi_flag,DM_2mtf,eDM_2mtf,2mtf_flag,DM_flat,eDM_flat,flat_flag
PGC,,,,,,,,,,,,,,,,,,,,,
2,73150,200275,4726,4785,34.51,0.46,0,34.51,0.46,0,...,-10000,-10000.00,-10000.00,-10000,-10000.0,-10000.0,-10000,-10000.0,-10000.0,-10000
4,120,202766,4109,4153,33.47,0.39,0,33.49,0.49,0,...,0,33.54,0.39,0,-10000.0,-10000.0,-10000,-10000.0,-10000.0,-10000
12,12,210177,6195,6296,34.97,0.41,0,34.97,0.41,0,...,-10000,-10000.00,-10000.00,-10000,-10000.0,-10000.0,-10000,-10000.0,-10000.0,-10000
16,16,211419,5312,5386,34.63,0.45,0,34.63,0.45,0,...,-10000,-10000.00,-10000.00,-10000,-10000.0,-10000.0,-10000,-10000.0,-10000.0,-10000
55,55,0,4454,4506,34.22,0.39,0,33.83,0.60,0,...,0,34.42,0.39,0,-10000.0,-10000.0,-10000,-10000.0,-10000.0,-10000


In [4]:
df = pd.read_csv('ALLcf3SNIa.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())

df['pgc'] = df['PGC']
df_ALLcf3SNIa = df.set_index('PGC')

df_ALLcf3SNIa["PGC1"] = df_ALLcf3SNIa.apply(lambda x: x.PGC1 if x.PGC1 != 0 else x.pgc, axis=1)


# df_ALLcf3SNIa['MINeDM'] = 0.10
# df_ALLcf3SNIa['min'] = df[["A","tmp"]].max(axis=1)


df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"Vcmb": "Vcmb_ALLcf3SNIa"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"PGC1": "PGC1_ALLcf3SNIa"})
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMav": "DM_ALLcf3SNIa"})
df_ALLcf3SNIa["eDM_ALLcf3SNIa"] = 0.15
df_ALLcf3SNIa["DM_ALLcf3SNIa"][df_ALLcf3SNIa["DM_ALLcf3SNIa"]==0] = np.nan
df_ALLcf3SNIa["eDM_ALLcf3SNIa"][df_ALLcf3SNIa["DM_ALLcf3SNIa"].isna()] = np.nan

#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_r"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_r"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMr": "DM_cf3SNIa_r"})
df_ALLcf3SNIa["eDMr"] = np.sqrt(df_ALLcf3SNIa["eDMr"]**2+0.15**2)
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMr": "eDM_cf3SNIa_r"})
df_ALLcf3SNIa["DM_cf3SNIa_r"][df_ALLcf3SNIa["DM_cf3SNIa_r"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_r"][df_ALLcf3SNIa["DM_cf3SNIa_r"].isna()] = np.nan

#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_w"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_w"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMw": "DM_cf3SNIa_w"})
df_ALLcf3SNIa["eDM_cf3SNIa_w"] = 0.15
df_ALLcf3SNIa["DM_cf3SNIa_w"][df_ALLcf3SNIa["DM_cf3SNIa_w"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_w"][df_ALLcf3SNIa["DM_cf3SNIa_w"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_u"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_u"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMu": "DM_cf3SNIa_u"})
df_ALLcf3SNIa["eDMu"] = np.sqrt(df_ALLcf3SNIa["eDMu"]**2+0.15**2)
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMu": "eDM_cf3SNIa_u"})
df_ALLcf3SNIa["DM_cf3SNIa_u"][df_ALLcf3SNIa["DM_cf3SNIa_u"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_u"][df_ALLcf3SNIa["DM_cf3SNIa_u"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_j"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_j"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMj": "DM_cf3SNIa_j"})
df_ALLcf3SNIa["eDMj"] = np.sqrt(df_ALLcf3SNIa["eDMj"]**2+0.15**2)
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMj": "eDM_cf3SNIa_j"})
df_ALLcf3SNIa["DM_cf3SNIa_j"][df_ALLcf3SNIa["DM_cf3SNIa_j"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_j"][df_ALLcf3SNIa["DM_cf3SNIa_j"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_p"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_p"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMp": "DM_cf3SNIa_p"})
df_ALLcf3SNIa["eDMp"] = np.sqrt(df_ALLcf3SNIa["eDMp"]**2+0.15**2)
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"eDMp": "eDM_cf3SNIa_p"})
df_ALLcf3SNIa["DM_cf3SNIa_p"][df_ALLcf3SNIa["DM_cf3SNIa_p"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_p"][df_ALLcf3SNIa["DM_cf3SNIa_p"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_h"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_h"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMh": "DM_cf3SNIa_h"})
df_ALLcf3SNIa["eDM_cf3SNIa_h"] = 0.15
df_ALLcf3SNIa["DM_cf3SNIa_h"][df_ALLcf3SNIa["DM_cf3SNIa_h"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_h"][df_ALLcf3SNIa["DM_cf3SNIa_h"].isna()] = np.nan
#################
df_ALLcf3SNIa["Vcmb_cf3SNIa_f"] = df_ALLcf3SNIa["Vcmb_ALLcf3SNIa"]
df_ALLcf3SNIa["PGC1_cf3SNIa_f"] = df_ALLcf3SNIa["PGC1_ALLcf3SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa.rename(columns={"DMf": "DM_cf3SNIa_f"})
df_ALLcf3SNIa["eDM_cf3SNIa_f"] = 0.15
df_ALLcf3SNIa["DM_cf3SNIa_f"][df_ALLcf3SNIa["DM_cf3SNIa_f"]==0] = np.nan
df_ALLcf3SNIa["eDM_cf3SNIa_f"][df_ALLcf3SNIa["DM_cf3SNIa_f"].isna()] = np.nan

# "ALLcf3SNIa", "cf3SNIa_w"
catalogs = ["cf3SNIa_r", "cf3SNIa_u", "cf3SNIa_j","cf3SNIa_p", "cf3SNIa_h","cf3SNIa_f", "cf3SNIa_w"]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]

    
df_ALLcf3SNIa["SNIa_ALLcf3SNIa"] = df_ALLcf3SNIa["SNIa"]
df_ALLcf3SNIa = df_ALLcf3SNIa[ss+["SNIa_ALLcf3SNIa"]]



print(len(df_ALLcf3SNIa))

df_ALLcf3SNIa.head()

389


<ipython-input-4-0a272f8ec33b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ALLcf3SNIa["DM_ALLcf3SNIa"][df_ALLcf3SNIa["DM_ALLcf3SNIa"]==0] = np.nan
<ipython-input-4-0a272f8ec33b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ALLcf3SNIa["eDM_ALLcf3SNIa"][df_ALLcf3SNIa["DM_ALLcf3SNIa"].isna()] = np.nan
<ipython-input-4-0a272f8ec33b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ALLcf3SNIa["DM_cf3SNIa_r"][df_ALLcf3SN

,PGC1_cf3SNIa_r,DM_cf3SNIa_r,eDM_cf3SNIa_r,Vcmb_cf3SNIa_r,PGC1_cf3SNIa_u,DM_cf3SNIa_u,eDM_cf3SNIa_u,Vcmb_cf3SNIa_u,PGC1_cf3SNIa_j,DM_cf3SNIa_j,...,Vcmb_cf3SNIa_h,PGC1_cf3SNIa_f,DM_cf3SNIa_f,eDM_cf3SNIa_f,Vcmb_cf3SNIa_f,PGC1_cf3SNIa_w,DM_cf3SNIa_w,eDM_cf3SNIa_w,Vcmb_cf3SNIa_w,SNIa_ALLcf3SNIa
PGC,,,,,,,,,,,,,,,,,,,,,
250,250,34.93,0.219317,6905,250,34.34,0.234307,6905,250,NaN,...,6905,250,NaN,NaN,6905,250,NaN,NaN,6905,2006sr
282,282,35.95,0.258070,11001,282,NaN,NaN,11001,282,NaN,...,11001,282,NaN,NaN,11001,282,NaN,NaN,11001,2008hj
415,415,34.97,0.226716,7183,415,34.44,0.242074,7183,415,NaN,...,7183,415,NaN,NaN,7183,415,NaN,NaN,7183,2003it
515,515,NaN,NaN,4897,515,33.70,0.258070,4897,515,NaN,...,4897,515,NaN,NaN,4897,515,NaN,NaN,4897,2002hw
963,963,NaN,NaN,3607,963,32.64,0.317648,3607,963,32.87,...,3607,963,NaN,NaN,3607,963,NaN,NaN,3607,1998dk


In [5]:
df = pd.read_csv('ScolnicSNIa3Tables.csv', delimiter='|')

df = df.rename(columns=lambda x: x.strip())
df["PGC1"] = df.apply(lambda x: x.PGC1 if x.PGC1 != 0 else x.PGC, axis=1)

df = df.set_index('PGC')



df = df.rename(columns={"Nest": "Nest_scolnic"})
df = df.rename(columns={"SNIa": "SNIa_scolnic"})
df["Vcmb_scolnic"] = Vh2V3k(df["Glon"], df["Glat"], df["Vhel"])
df = df.rename(columns={"PGC1": "PGC1_scolnic"})
df["eDM_scolnic"] = df.apply(lambda x: 
                             x.eDMp if x["Tab"]==1 
                             else (x["eDM18"] if 
                            ["Tab"]==2 else x["eDM17"]) 
                            , axis=1)


df["eDM_scolnic"] = np.sqrt(df["eDM_scolnic"]**2+0.15**2)

df_scolnic = df.rename(columns={"DM": "DM_scolnic"})

df_scolnic["DM_scolnic"][df_scolnic["DM_scolnic"]==0] = np.nan
df_scolnic["eDM_scolnic"][df_scolnic["DM_scolnic"].isna()] = np.nan
df_scolnic.head()

<ipython-input-5-bbc9e56cc2de>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scolnic["DM_scolnic"][df_scolnic["DM_scolnic"]==0] = np.nan
<ipython-input-5-bbc9e56cc2de>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scolnic["eDM_scolnic"][df_scolnic["DM_scolnic"].isna()] = np.nan


,SNIa_scolnic,PGC1_scolnic,Nest_scolnic,Tab,DM_scolnic,DMp,eDMp,DM18,eDM18,DM17,eDM17,Vhel,Vsn,Glon,Glat,SGL,SGB,Vcmb_scolnic,eDM_scolnic
PGC,,,,,,,,,,,,,,,,,,,
250,2006sr,250,208532,1,35.061,35.061,0.133,35.057,0.102,35.046,0.079,7253,7147,108.8693,-38.3606,316.3211,17.6720,6905.301355,0.200472
282,2008hj,282,0,3,36.025,0.000,0.000,0.000,0.000,36.025,0.091,11360,11003,85.7160,-70.5385,282.0792,8.9703,11016.755644,0.175445
415,2003it,415,200542,1,35.150,35.150,0.140,35.147,0.108,35.149,0.089,7522,7582,110.6597,-34.3329,320.8087,17.8681,7183.381394,0.205183
963,1998dk,963,211877,1,33.339,33.339,0.191,33.350,0.168,33.366,0.114,3961,3625,102.8519,-62.1619,292.9641,9.5361,3606.990039,0.242860
1371,AT2016eoa,1351,200045,2,34.736,0.000,0.000,34.736,0.133,34.758,0.088,6227,6326,113.8635,-39.9082,316.3366,13.5013,5884.878602,0.173908


In [6]:
df = pd.read_csv('Scolnic2020.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df["PGC1"] = df.apply(lambda x: x.PGC1 if x.PGC1 != 0 else x.PGC, axis=1)

df = df.set_index('PGC')

df = df.rename(columns={"Nest": "Nest_scol20"})
df = df.rename(columns={"Vcmb": "Vcmb_scol20"})
df = df.rename(columns={"PGC1": "PGC1_scol20"})
df = df.rename(columns={"eDM": "eDM_scol20"})
df = df.rename(columns={"SNIa": "SNIa_scol20"})

df["eDM_scol20"] = np.sqrt(df["eDM_scol20"]**2+0.15**2)

df_scol20 = df.rename(columns={"DM": "DM_scol20"})
df_scol20["DM_scol20"][df_scol20["DM_scol20"]==0] = np.nan
df_scol20["eDM_scol20"][df_scol20["DM_scol20"].isna()] = np.nan
df_scol20.head()

df_scol20.head()

<ipython-input-6-48358cff9090>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scol20["DM_scol20"][df_scol20["DM_scol20"]==0] = np.nan
<ipython-input-6-48358cff9090>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scol20["eDM_scol20"][df_scol20["DM_scol20"].isna()] = np.nan


,SNIa_scol20,PGC1_scol20,Nest_scol20,DM_scol20,eDM_scol20,Vrc3,Vhel,Vcmb_scol20,eV,Vpec,...,De,J2000,Glon,Glat,SGL,SGB,EBV,Ty,lgMass,N
PGC,,,,,,,,,,,,,,,,,,,,,
250,2006sr,250,208532,35.0397,0.246891,7260,7270,6916,3,240,...,23.196171,J000335.0+231203,108.8693,-38.3607,316.3211,17.6719,5.6,0.08,10.7951,1
282,2008hj,282,0,35.9417,0.237162,11360,11275,10921,45,132,...,-11.168750,J000401.5-111027,85.7160,-70.5384,282.0792,8.9703,1.3,0.02,9.9461,2
415,2003it,415,200542,35.1078,0.248483,7516,7513,7165,3,322,...,27.449444,J000548.4+272657,110.6598,-34.3328,320.8089,17.8681,3.3,0.05,10.4235,1
963,1998dk,963,211877,33.3500,0.279551,3953,3945,3589,3,47,...,-0.736360,J001431.9-004415,102.8518,-62.1619,292.9641,9.5362,5.2,0.03,9.4149,1
1371,AT2016eoa,1351,200045,34.7503,0.260108,6227,6323,5975,21,391,...,22.435440,J002122.5+222601,113.8635,-39.9080,316.3368,13.5013,-4.8,0.06,10.2163,1


In [7]:
df = pd.read_csv('CSP1Burns18.csv', delimiter='|')

df = df.rename(columns=lambda x: x.strip())
df["PGC1"] = df.apply(lambda x: x.PGC1 if x.PGC1 != 0 else x.PGC, axis=1)
df = df.set_index('PGC')

df = df.rename(columns={"Nest": "Nest_csp1"})
df = df.rename(columns={"PGC1": "PGC1_csp1"})
df = df.rename(columns={"SNIa": "SNIa_csp1"})

df["Vcmb_csp1"] = df["z3k"] * 299792.458

df = df.rename(columns={"muCV": "DM_csp1"})

df["emu"] = np.sqrt(df["emu"]**2+0.15**2)

df_csp1 = df.rename(columns={"emu": "eDM_csp1"})

df_csp1["DM_csp1"][df_csp1["DM_csp1"]==0] = np.nan
df_csp1["eDM_csp1"][df_csp1["DM_csp1"].isna()] = np.nan

df_csp1.head()

<ipython-input-7-2cbc550f9c6c>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csp1["DM_csp1"][df_csp1["DM_csp1"]==0] = np.nan
<ipython-input-7-2cbc550f9c6c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csp1["eDM_csp1"][df_csp1["DM_csp1"].isna()] = np.nan


,SNIa_csp1,PGC1_csp1,Nest_csp1,DM_csp1,eDM_csp1,zh,z3k,sBV,esBV,Dm15,...,eRv,covER,Q,RA,Dec,glon,glat,sgl,sgb,Vcmb_csp1
PGC,,,,,,,,,,,,,,,,,,,,,
282,2008hj,282,0,36.11,0.155242,0.0379,0.0367,1.010,0.031,0.964,...,1.0,-0.0072,0.97,1.0080,-11.1687,85.2573,-70.7713,281.8123,8.8821,11002.383209
1583,2007A,1583,210125,34.20,0.161555,0.0176,0.0165,1.012,0.032,0.954,...,0.3,-0.0053,0.97,6.3194,12.8868,113.0917,-49.4843,307.0303,10.4751,4946.575557
2332,2006ej,2331,202847,34.67,0.161555,0.0204,0.0193,0.829,0.035,1.368,...,1.1,-0.0102,0.97,9.7490,-9.0157,113.1135,-71.6864,286.6520,1.3636,5785.994439
2559,2006et,2559,201302,34.66,0.158114,0.0222,0.0212,1.093,0.031,0.887,...,0.3,-0.0026,0.95,10.6909,-23.5584,85.8379,-86.7274,272.0395,-3.7542,6355.600110
3444,2007nq,3342,200043,36.19,0.155242,0.0450,0.0439,0.745,0.030,1.491,...,1.2,-0.0037,0.94,14.3900,-1.3887,126.5363,-64.8731,294.6544,-1.1653,13160.888906


In [8]:
df = pd.read_csv('lossGanesh13.csv', delimiter='|')
df["PGC1"] = df.apply(lambda x: x.PGC1 if x.PGC1 != 0 else x.PGC, axis=1)
df = df.set_index('PGC')

df = df.rename(columns={"Nest": "Nest_ganesh"})
df = df.rename(columns={"PGC1": "PGC1_ganesh"})

df["SNIa"] = df["SNIa"].apply(lambda x: x.split(" ")[1] if x != None else None)
df = df.rename(columns={"SNIa": "SNIa_ganesh"})

df["Vcmb_ganesh"] = df["z3k"] * 299792.458

df = df.rename(columns={"mu": "DM_ganesh"})
df["emu"] = np.sqrt(df["emu"]**2+0.15**2)

df_ganesh = df.rename(columns={"emu": "eDM_ganesh"})

df_ganesh["DM_ganesh"][df_ganesh["DM_ganesh"]==0] = np.nan
df_ganesh["eDM_ganesh"][df_ganesh["DM_ganesh"].isna()] = np.nan

df_ganesh.head()

<ipython-input-8-b0014bdd280e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ganesh["DM_ganesh"][df_ganesh["DM_ganesh"]==0] = np.nan
<ipython-input-8-b0014bdd280e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ganesh["eDM_ganesh"][df_ganesh["DM_ganesh"].isna()] = np.nan


,SNIa_ganesh,PGC1_ganesh,Nest_ganesh,RA,Dec,z3k,B,eB,x1,ex1,c,ec,DM_ganesh,eDM_ganesh,Sample,Ref,Vcmb_ganesh
PGC,,,,,,,,,,,,,,,,,
250,2006sr,250,208532,00 03 35.02,+23 11 46.2,0.023,16.123,0.030,-1.220,0.142,-0.020,0.029,35.070,0.226716,Low-z,5,6895.226534
465,2000dg,616,200076,00 06 13.81,+08 53 17.5,0.037,16.989,0.068,-1.442,0.420,-0.057,0.045,36.021,0.236620,Low-z,2,11092.320946
515,2002hw,515,210067,00 06 49.06,+08 37 48.5,0.016,16.646,0.038,-1.849,0.203,0.377,0.034,34.245,0.249201,Low-z,5,4796.679328
2332,2006ej,2331,202847,00 38 59.77,-09 00 56.6,0.019,15.743,0.031,-1.583,0.116,-0.035,0.030,34.683,0.235847,Low-z,2,5696.056702
2559,2006et,2559,201302,00 42 45.82,-23 33 30.4,0.021,15.950,0.034,0.812,0.170,0.110,0.030,34.781,0.232011,Low-z,5,6295.641618


In [9]:
how = 'outer'

df = df_ALLcf3SNIa.join(df_scol20, lsuffix='_l', rsuffix='_r', how=how)
df = df.join(df_scolnic, lsuffix='_l', rsuffix='_r', how=how)
df = df.join(df_csp1, lsuffix='_l1', rsuffix='_r1', how=how)
df = df.join(df_ganesh, lsuffix='_l2', rsuffix='_r2', how=how)

# catalogs = ["scol20"]

catalogs = ["scol20", "scolnic", "csp1", "ganesh"]
catalogs += ["cf3SNIa_r", "cf3SNIa_u", "cf3SNIa_j","cf3SNIa_p", "cf3SNIa_h","cf3SNIa_f"]#, "cf3SNIa_w"]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]


sn = []
catalogs_sn = ["scol20", "scolnic", "csp1", "ganesh", "ALLcf3SNIa"]    
for cat in catalogs_sn:
    sn += ['SNIa_'+cat]    
    
    
df = df[ss+sn]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat]   
df['pgc1'] = df[ss].median(axis=1)
df = df[~df['pgc1'].isna()]

print(len(df))

df.head()

841


,PGC1_scol20,DM_scol20,eDM_scol20,Vcmb_scol20,PGC1_scolnic,DM_scolnic,eDM_scolnic,Vcmb_scolnic,PGC1_csp1,DM_csp1,...,PGC1_cf3SNIa_f,DM_cf3SNIa_f,eDM_cf3SNIa_f,Vcmb_cf3SNIa_f,SNIa_scol20,SNIa_scolnic,SNIa_csp1,SNIa_ganesh,SNIa_ALLcf3SNIa,pgc1
PGC,,,,,,,,,,,,,,,,,,,,,
250,250.0,35.0397,0.246891,6916.0,250.0,35.061,0.200472,6905.301355,NaN,NaN,...,250.0,NaN,NaN,6905.0,2006sr,2006sr,NaN,2006sr,2006sr,250.0
282,282.0,35.9417,0.237162,10921.0,282.0,36.025,0.175445,11016.755644,282.0,36.11,...,282.0,NaN,NaN,11001.0,2008hj,2008hj,2008hj,NaN,2008hj,282.0
415,415.0,35.1078,0.248483,7165.0,415.0,35.150,0.205183,7183.381394,NaN,NaN,...,415.0,NaN,NaN,7183.0,2003it,2003it,NaN,NaN,2003it,415.0
465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000dg,NaN,616.0
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515.0,NaN,NaN,4897.0,NaN,NaN,NaN,2002hw,2002hw,515.0


In [10]:
def unique_SNIa(X):
    
    l = []
    for x in X:
        if not x in l and not x is np.nan:
            l.append(x)
    s = ''
    for m in l[:-1]:
        if not m is None:
            s = s+ str(m)+' / ' 
            
    if not l[-1] is None:
        s += str(l[-1])
    return s
    


df["SNIa"] = df[sn].apply(unique_SNIa, axis=1)

df.tail(20)

,PGC1_scol20,DM_scol20,eDM_scol20,Vcmb_scol20,PGC1_scolnic,DM_scolnic,eDM_scolnic,Vcmb_scolnic,PGC1_csp1,DM_csp1,...,DM_cf3SNIa_f,eDM_cf3SNIa_f,Vcmb_cf3SNIa_f,SNIa_scol20,SNIa_scolnic,SNIa_csp1,SNIa_ganesh,SNIa_ALLcf3SNIa,pgc1,SNIa
PGC,,,,,,,,,,,,,,,,,,,,,
9003172,NaN,NaN,NaN,NaN,9003172.0,37.222,0.167260,-25.626802,NaN,NaN,...,NaN,NaN,NaN,NaN,CSS151120-0,NaN,NaN,NaN,9003172.0,CSS151120-0
9003173,9003173.0,37.3889,0.244831,20509.0,9003173.0,37.454,0.170473,209.577353,NaN,NaN,...,NaN,NaN,NaN,CSS160129,CSS160129,NaN,NaN,NaN,9003173.0,CSS160129
9003174,NaN,NaN,NaN,NaN,9003174.0,36.522,0.350698,-281.459602,NaN,NaN,...,NaN,NaN,NaN,NaN,CSS161013,NaN,NaN,NaN,9003174.0,CSS161013
9003177,3091040.0,36.2974,0.247288,10796.0,9003177.0,36.255,0.165109,-314.163968,NaN,NaN,...,NaN,NaN,NaN,PS15bbn,PS15bbn,NaN,NaN,NaN,6047108.5,PS15bbn
9003178,9003178.0,37.6383,0.236002,23549.0,9003178.0,37.662,0.163878,-228.581823,NaN,NaN,...,NaN,NaN,NaN,PS15bif,PS15bif,NaN,NaN,NaN,9003178.0,PS15bif
9003179,NaN,NaN,NaN,NaN,9003179.0,38.192,0.203823,-346.326978,NaN,NaN,...,NaN,NaN,NaN,NaN,PS15bst,NaN,NaN,NaN,9003179.0,PS15bst
9003180,9003180.0,37.4026,0.245464,21432.0,9003180.0,37.325,0.179725,-332.637139,NaN,NaN,...,NaN,NaN,NaN,PS15bwh,PS15bwh,NaN,NaN,NaN,9003180.0,PS15bwh
9003181,NaN,NaN,NaN,NaN,9003181.0,38.632,0.175966,-310.097303,NaN,NaN,...,NaN,NaN,NaN,NaN,PS15cfn,NaN,NaN,NaN,9003181.0,PS15cfn
9003183,61506.0,35.3009,0.248722,7180.0,61506.0,35.358,0.163478,-105.460741,NaN,NaN,...,NaN,NaN,NaN,PS16ccn,PS16ccn,NaN,NaN,NaN,61506.0,PS16ccn


In [11]:
for cat in catalogs:
    df['Vmod_'+cat] = Vcmb2Vmod(df['Vcmb_'+cat])
    logD = (df['DM_'+cat]-25)/5.
    df['logH_'+cat] = np.log10(df['Vmod_'+cat]) - logD 
    
    print(cat, 10**df['logH_'+cat][df['Vmod_'+cat]>4000].median())

scol20 70.07830412210477
scolnic 69.80422461999798
csp1 71.8217647081815
ganesh 69.71910030320372
cf3SNIa_r 75.01347474759342
cf3SNIa_u 98.8497425557428
cf3SNIa_j 100.95288477435838
cf3SNIa_p 100.78633017284425
cf3SNIa_h 98.54251929336785
cf3SNIa_f 98.7488105588484


/home/ehsan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ehsan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ehsan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ehsan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ehsan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ehsan/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning

In [12]:
df["PGC1_cf4"] = df["PGC1_scol20"]

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
print(len(df["PGC1_cf4"][ind]))

for cat in catalogs:
    ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
    df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
    

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
print(len(df["PGC1_cf4"][ind]))

ind = (df["PGC1_cf4"].isna()) | (df["PGC1_cf4"]==0)
df["PGC1_cf4"][ind] = np.nan

<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]


202
0


<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PGC1_cf4"][ind] = df["PGC1_"+cat][ind]
<ipython-input-12-c5d49ab19b3c>:8: SettingWithCopyWarning: 
A value

In [13]:
df["Vmod_cf4"] = df["Vmod_scol20"]

ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
print(len(df["Vmod_cf4"][ind]))

for cat in catalogs:
    ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
    df["Vmod_cf4"][ind] = df["Vmod_"+cat][ind]
    
ind = (df["Vmod_cf4"].isna()) | (df["Vmod_cf4"]==0)
df["Vmod_cf4"][ind] = np.nan

202


<ipython-input-13-dc447073f27d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vmod_cf4"][ind] = df["Vmod_"+cat][ind]
<ipython-input-13-dc447073f27d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vmod_cf4"][ind] = df["Vmod_"+cat][ind]
<ipython-input-13-dc447073f27d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vmod_cf4"][ind] = df["Vmod_"+cat][ind]
<ipython-input-13-dc447073f27d>:8: SettingWithCopyWarning: 
A value

In [14]:
df["Vcmb_cf4"] = df["Vcmb_scol20"]

ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
print(len(df["Vcmb_cf4"][ind]))

for cat in catalogs:
    ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
    df["Vcmb_cf4"][ind] = df["Vcmb_"+cat][ind]
    
ind = (df["Vcmb_cf4"].isna()) | (df["Vcmb_cf4"]==0)
df["Vcmb_cf4"][ind] = np.nan

202


<ipython-input-14-1558c78ba75a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vcmb_cf4"][ind] = df["Vcmb_"+cat][ind]
<ipython-input-14-1558c78ba75a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vcmb_cf4"][ind] = df["Vcmb_"+cat][ind]
<ipython-input-14-1558c78ba75a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Vcmb_cf4"][ind] = df["Vcmb_"+cat][ind]
<ipython-input-14-1558c78ba75a>:8: SettingWithCopyWarning: 
A value

In [15]:
df.head()

,PGC1_scol20,DM_scol20,eDM_scol20,Vcmb_scol20,PGC1_scolnic,DM_scolnic,eDM_scolnic,Vcmb_scolnic,PGC1_csp1,DM_csp1,...,logH_cf3SNIa_j,Vmod_cf3SNIa_p,logH_cf3SNIa_p,Vmod_cf3SNIa_h,logH_cf3SNIa_h,Vmod_cf3SNIa_f,logH_cf3SNIa_f,PGC1_cf4,Vmod_cf4,Vcmb_cf4
PGC,,,,,,,,,,,,,,,,,,,,,
250,250.0,35.0397,0.246891,6916.0,250.0,35.061,0.200472,6905.301355,NaN,NaN,...,NaN,7030.895465,NaN,7030.895465,1.977011,7030.895465,NaN,250.0,7042.295404,6916.000000
282,282.0,35.9417,0.237162,10921.0,282.0,36.025,0.175445,11016.755644,282.0,36.11,...,NaN,11319.146989,NaN,11319.146989,NaN,11319.146989,NaN,282.0,11234.563764,10921.000000
415,415.0,35.1078,0.248483,7165.0,415.0,35.150,0.205183,7183.381394,NaN,NaN,...,NaN,7319.196034,NaN,7319.196034,1.966463,7319.196034,NaN,415.0,7300.516924,7165.000000
465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,616.0,11415.739891,11092.320946
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4960.457172,NaN,4960.457172,1.959522,4960.457172,NaN,515.0,4857.569706,4796.679328


In [16]:
myDict = {}

myDict["scol20"] = 0.
myDict["scolnic"] = -0.019
myDict["csp1"] = 0.055
myDict["ganesh"] = 0.001
myDict["cf3SNIa_r"] = 0.130
myDict["cf3SNIa_u"] = 0.708
myDict["cf3SNIa_j"] = 0.717
myDict["cf3SNIa_p"] = 0.723
myDict["cf3SNIa_h"] = 0.705
myDict["cf3SNIa_f"] = 0.701


for cat in catalogs:
    df['DM_'+cat] += myDict[cat]
    df['w_'+cat] = (1.-0)/df['eDM_'+cat]**2
    df['w_'+cat] = df['w_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['e_'+cat] = (1.-0)*df['eDM_'+cat]
    df['e_'+cat] = df['e_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['xw_'+cat] = (df['DM_'+cat])*df['w_'+cat]

sw = ['w_'+cat for cat in catalogs]
ee = ['e_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)

df["eDM_av"] = df[ee].min(axis=1) # np.sqrt(Err2)
df["DM_av"] = df[sx].sum(axis=1)*Err2    

In [17]:
df['av_flag'] = 0*df['DM_av']
df['av_flag'][df['DM_av'].isna()] = 1

<ipython-input-17-d6a2f099f968>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['av_flag'][df['DM_av'].isna()] = 1


In [18]:
catalogs

['scol20',
 'scolnic',
 'csp1',
 'ganesh',
 'cf3SNIa_r',
 'cf3SNIa_u',
 'cf3SNIa_j',
 'cf3SNIa_p',
 'cf3SNIa_h',
 'cf3SNIa_f']

In [19]:
df_old = pd.read_csv('CF4_SNIa_amou_old.csv', delimiter='|')
df_old = df_old[["IDT14g"]]
df = df.join(df_old, lsuffix='_new', rsuffix='_old', how='left')

df.columns

Index(['PGC1_scol20', 'DM_scol20', 'eDM_scol20', 'Vcmb_scol20', 'PGC1_scolnic',
       'DM_scolnic', 'eDM_scolnic', 'Vcmb_scolnic', 'PGC1_csp1', 'DM_csp1',
       ...
       'w_cf3SNIa_h', 'e_cf3SNIa_h', 'xw_cf3SNIa_h', 'w_cf3SNIa_f',
       'e_cf3SNIa_f', 'xw_cf3SNIa_f', 'eDM_av', 'DM_av', 'av_flag', 'IDT14g'],
      dtype='object', length=104)

In [20]:
ss = ['SNIa', 'IDT14g', 'PGC1_cf4', 'Vcmb_cf4', 'Vmod_cf4', "DM_av", "eDM_av"]
for cat in catalogs:
    ss += ["DM_"+cat, "eDM_"+cat]

df = df.fillna(0)

df = df[ss]


df = df.rename(columns={'PGC1_cf4':'PGC1',
                        'Nest_cf4':'Nest',
                        'Vcmb_cf4': 'Vcmb',
                        'Vmod_cf4': 'Vmod'
                       })

In [21]:
for cat in catalogs:
    edm = 'eDM_'+cat
    dm = 'DM_'+cat
    
    print(dm, edm)
    
    df[edm] = df.apply(lambda x: x[edm] if x[dm]!=0 else np.nan, axis=1)
    df[dm] = df.apply(lambda x: x[dm] if x[dm]!=0 else np.nan, axis=1)

DM_scol20 eDM_scol20
DM_scolnic eDM_scolnic
DM_csp1 eDM_csp1
DM_ganesh eDM_ganesh
DM_cf3SNIa_r eDM_cf3SNIa_r
DM_cf3SNIa_u eDM_cf3SNIa_u
DM_cf3SNIa_j eDM_cf3SNIa_j
DM_cf3SNIa_p eDM_cf3SNIa_p
DM_cf3SNIa_h eDM_cf3SNIa_h
DM_cf3SNIa_f eDM_cf3SNIa_f


In [22]:
len(df)

841

In [23]:
df = df[df["DM_av"]!=0]

In [24]:
len(df)

815

In [25]:
df.to_csv("SNIa_CF4_components_bar.csv", sep='|')

In [26]:
table   = np.genfromtxt('SNIa_CF4_components_bar.csv' , delimiter='|', 
                        filling_values=-10000, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["SNIa"]:
        myTable.add_column(Column(data=table[key], name=key))
    elif key in ["PGC", "PGC1", "IDT14g"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.2f'))

## to be used on EDD
myTable.write('SNIa_CF4_components_bar', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)